# 데이터 수집

- 유튜브 플레이 리스트 제목 : 조회수 100만 이상의 즉문즉설 베스트
- 유뷰브 플레이 리스트 ID : `PLGiaCgd9PatcGBfZ7xTGdTAsHoNPRQ_AP`

In [2]:
! yt-dlp --write-auto-sub --sub-lang ko --skip-download -o "data/%(title)s.%(ext)s" "https://www.youtube.com/playlist?list=PLGiaCgd9PatcGBfZ7xTGdTAsHoNPRQ_AP"

[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLGiaCgd9PatcGBfZ7xTGdTAsHoNPRQ_AP
[youtube:tab] PLGiaCgd9PatcGBfZ7xTGdTAsHoNPRQ_AP: Downloading webpage
[youtube:tab] PLGiaCgd9PatcGBfZ7xTGdTAsHoNPRQ_AP: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: 조회수 100만 이상의 즉문즉설 베스트 베스트
[youtube:tab] PLGiaCgd9PatcGBfZ7xTGdTAsHoNPRQ_AP page 1: Downloading API JSON
[youtube:tab] PLGiaCgd9PatcGBfZ7xTGdTAsHoNPRQ_AP page 2: Downloading API JSON
[youtube:tab] Playlist 조회수 100만 이상의 즉문즉설 베스트 베스트: Downloading 273 items of 273
[download] Downloading item 1 of 273
[youtube] Extracting URL: https://www.youtube.com/watch?v=pMevsZErXr4
[youtube] pMevsZErXr4: Downloading webpage
[youtube] pMevsZErXr4: Downloading tv client config
[youtube] pMevsZErXr4: Downloading tv player API JSON
[youtube] pMevsZErXr4: Downloading ios player API JSON
[youtube] pMevsZErXr4: Downloading m3u8 information
[info] pMevsZErXr4: Downloading subtitles: ko
[info] pMevsZErXr4

ERROR: [youtube] jybBMfI5ffQ: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
ERROR: [youtube] 91UmNJ7jmUo: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
ERROR: [youtube] x9XGbwJ0cXo: Private video. Sign in if you've been granted access to this video. Use --cookies-from-browser or --cookies for the authentication. See  https://g

# 데이터 가공

In [ ]:
import os
import re
from glob import glob
from pathlib import Path

def clean_vtt_with_dedup(filepath):
    if type(filepath) == str:
        with open(filepath, 'r', encoding='utf-8') as f:
            docs = [f.readlines()]

    elif type(filepath) == list:
        docs = []
        for file in filepath:
            with open(file, 'r', encoding='utf-8') as f:
                docs.append(f.readlines())
    else:
        raise TypeError("filepath must be str or list")

    cleaned_lines = []
    buffer = ""
    prev_line = ""

    for lines in docs:
        for line in lines:
            line = line.strip()

            # 타임라인 줄은 무시
            if re.match(r"^\d\d:\d\d:\d\d\.\d+ -->", line):
                continue

            # HTML 태그 및 시간태그 제거
            line = re.sub(r"<\d\d:\d\d:\d\d\.\d+><c>", "", line)
            line = re.sub(r"</c>", "", line)
            line = line.strip()

            if not line:
                continue

            # 중복 제거: 이전 줄이 포함되어 있으면 제거
            if line == prev_line:
                continue
            elif line.startswith(prev_line) and prev_line:
                # 현재 줄이 이전 줄을 포함한 확장형이면 이전 줄 제거
                if cleaned_lines:
                    cleaned_lines.pop()
            cleaned_lines.append(line)
            prev_line = line

    return "\n".join(cleaned_lines)

# vtt 데이터 가공
text = clean_vtt_with_dedup(glob(os.path.join('data', '*.vtt')))
text

TypeError: unhashable type: 'list'

In [ ]:
import os
import re

def clean_filename(filename):
    # 괄호 및 내부 내용 제거: (), [], {}
    return re.sub(r"[\[\(\{].*?[\]\)\}]", "", filename).strip()

def rename_vtt_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".vtt") or filename.endswith(".srt"):
            new_name = clean_filename(filename)
            # 중복 방지: 확장자는 그대로 유지
            new_name = re.sub(r"\s+", " ", new_name)  # 공백 정리
            new_name = new_name.rstrip(".") + os.path.splitext(filename)[1]
            os.rename(
                os.path.join(folder_path, filename),
                os.path.join(folder_path, new_name)
            )
            print(f"Renamed: {filename} → {new_name}")

# 사용 예시
rename_vtt_files("./subtitles")  # 자막 파일 있는 폴더 경로
